In [1]:
import cv2
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt


In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture("../test/30CST.mp4")
ret, old_frame = cap.read()
if not ret:
    print("Failed to load video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}
# Initialize lists to store keypoint data for displacement and velocity analysis
keypoints_history = []
displacements = [] #in pixels
velocities = [] #in pixels per second

# Get FPS of the video
fps = cap.get(cv2.CAP_PROP_FPS)
time_interval = 1 / fps
print(f"Video FPS: {fps}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        cap.release()
        
        break

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process image for pose
    results = pose.process(rgb_image)

    # Convert back to BGR for visualization, important for other CV operations to work
    annotated_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        # Extract keypoints
        landmarks = results.pose_landmarks.landmark
        hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * frame.shape[1]),
               int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * frame.shape[0])]
        knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * frame.shape[0])]
        ankle = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * frame.shape[1]),
                 int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * frame.shape[0])]
        keypoints = np.array([hip, knee, ankle], dtype=np.float32).reshape(-1, 1, 2)
        # keypoints is a 3D array of shape (3, 1, 2) where each element is a 2D array of shape (1, 2)
        # [
        #     [[x_hip, y_hip]],
        #     [[x_knee, y_knee]],
        #     [[x_ankle, y_ankle]]
        # ]

        # Optical flow
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, keypoints, None, **lk_params)
        
        if keypoints_history:
            last_keypoints = keypoints_history[-1]
            for i, (new, old) in enumerate(zip(p1, last_keypoints)):
                displacement = np.linalg.norm(new - old)
                displacements.append(displacement)
                if len(displacements) > 1:
                    velocity = (displacements[-1] - displacements[-2]) / time_interval 
                    velocities.append(velocity)

        keypoints_history.append(p1.copy())
        
        # Draw optical flow tracks
        for i, (new, old) in enumerate(zip(p1, keypoints)):
            a, b = new.ravel().astype(int)
            c, d = old.ravel().astype(int)
            cv2.line(annotated_image, (a, b), (c, d), (0, 255, 0), 2)
            cv2.circle(annotated_image, (a, b), 5, (0, 0, 255), -1)

        # Update previous frame and points for next iteration
        old_gray = frame_gray.copy()
        keypoints = p1.copy()

    # Display the frame
    cv2.imshow('Mediapipe Feed', annotated_image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1715790970.687597 10306981 gl_context.cc:357] GL version: 2.1 (2.1 ATI-5.2.4), renderer: AMD Radeon Pro 5500M OpenGL Engine


Video FPS: 30.0


In [6]:
# keypoints_history
keypoints
# displacements
# velocities

array([[[ 905.94904, 1431.8617 ]],

       [[ 596.29346, 1446.9176 ]],

       [[ 599.96277, 1754.2441 ]]], dtype=float32)